In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from config import pw

In [2]:
data_path= 'Resources/Petroleum_Well.csv'
wells_df=pd.read_csv(data_path)

In [3]:
#Converting csv file into Data Frame
wells_df=pd.DataFrame(wells_df).copy()
wells_df

,X,Y,OBJECTID,OGF_ID,WELL_ID,LICENCE_NUMBER,WELL_NAME,OPERATOR,WELL_TYPE,WELL_MODE,...,DEEPENING_FLG,REENTRY_FLG,LATERAL_FLG,SPUD_DATE,TOTAL_DEPTH_REACHED_DATE,PLUGGING_END_DATE,MAP_SYMBOL_CODE,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME
0,-80.218198,43.133536,130116593,244130641,32702,F020287,Bow Park Farm Well #9,Bow Park Farms,Natural Gas Well,Unknown,...,N,N,N,1904/01/01 00:00:00+00,1904/01/01 00:00:00+00,1970/01/01 00:00:00+00,GP - UNK,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
1,-80.206309,43.121120,130116609,244130642,32703,F020288,Bow Park Farm Well #3,Bow Park Farms,Natural Gas Well,Unknown,...,N,N,N,1904/01/01 00:00:00+00,1904/01/01 00:00:00+00,1970/01/01 00:00:00+00,GP - UNK,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
2,-80.213643,43.113897,130116625,244130643,32704,F020289,Bow Park Farm Well #6,Bow Park Farms,Natural Gas Well,Unknown,...,N,N,N,1904/01/01 00:00:00+00,1904/01/01 00:00:00+00,1970/01/01 00:00:00+00,GP - UNK,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
3,-80.337281,43.177397,130116641,244130644,32705,F020290,Brant Hydro Station,Unknown,Dry Hole,Unknown,...,N,N,N,1800/01/01 00:00:00+00,1800/01/01 00:00:00+00,1970/01/01 00:00:00+00,DH - UNK,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
4,-80.209004,43.133897,130116657,244130645,32706,F020291,Bow Park Well #7,Bow Park Farms,Gas Show,Unknown,...,N,N,N,1800/01/01 00:00:00+00,1800/01/01 00:00:00+00,1970/01/01 00:00:00+00,GS - UNK,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26661,-82.159344,42.110886,130543169,244157302,9034,T000165,C.W.P. - L.E. MLO No.25,Lagasco Inc.,Natural Gas Well,Active Well,...,N,N,N,1959/08/06 00:00:00+00,1959/09/19 00:00:00+00,1970/01/01 00:00:00+00,GP - ACT,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
26662,-82.161011,42.101720,130543185,244157303,9035,T000169,C.W.P. - L.E. MLO No.26,Lagasco Inc.,Natural Gas Well,Active Well,...,N,N,N,1959/09/13 00:00:00+00,1959/10/09 00:00:00+00,1970/01/01 00:00:00+00,GP - ACT,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
26663,-82.094620,42.133109,130543201,244157304,9036,T000535,C.W.P. - L.E. MLO No.27,Dundee Oil and Gas Limited,Natural Gas Well,Abandoned Well,...,N,N,N,1960/06/07 00:00:00+00,1960/07/04 00:00:00+00,2017/06/06 00:00:00+00,GP - ABD,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00
26664,-79.872912,42.944242,130543217,244157305,9037,T005777,(private well name),(private operator),Private Gas Well,Active Well,...,N,N,N,1981/11/30 00:00:00+00,1982/01/18 00:00:00+00,1970/01/01 00:00:00+00,PGP - ACT,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00,2020/09/23 14:27:56+00


In [4]:
#geting unique values fo well_mode and creating a Data Frame with them
well_mode= wells_df['WELL_MODE'].unique()
cat_well_mode=pd.DataFrame(well_mode)
cat_well_mode.reset_index(inplace=True)
cat_well_mode.columns=['well_mode_id','well_mode']
cat_well_mode.head()

,well_mode_id,well_mode
0,0,Unknown
1,1,Abandoned Well
2,2,Suspended Well
3,3,Active Well
4,4,Capped Well


In [5]:
wells_df['OPERATOR']=wells_df['OPERATOR'].str.upper()
operator= wells_df['OPERATOR'].unique()
cat_operator=pd.DataFrame(operator)
cat_operator.reset_index(inplace=True) 
cat_operator.columns=['operator_id','operator_name']
cat_operator

,operator_id,operator_name
0,0,BOW PARK FARMS
1,1,UNKNOWN
2,2,DOWNSTOETTER
3,3,DOMINION NATURAL GAS CO. LTD.
4,4,UNITED OIL CO.
...,...,...
2197,2197,"MASTRONARDI, JOE"
2198,2198,"MURPHY, J. W."
2199,2199,"APPLEMAN, J.C."
2200,2200,"ROWE, JOHN MICHAEL"


In [6]:
well_type= wells_df['WELL_TYPE'].unique()
cat_well_type=pd.DataFrame(well_type)
cat_well_type.reset_index(inplace=True)
cat_well_type.columns=['well_type_id','well_type']
cat_well_type

,well_type_id,well_type
0,0,Natural Gas Well
1,1,Dry Hole
2,2,Gas Show
3,3,Oil Well
4,4,Location
5,5,Brine Well
6,6,Oil Show
7,7,Stratigraphic Test
8,8,Oil and Gas Well
9,9,Historical Oil Well


In [45]:
well_hdr = wells_df[['WELL_ID','X','Y', 'WELL_NAME','LICENCE_NUMBER',
                  'TOTAL_DEPTH','SPUD_DATE','OPERATOR','WELL_TYPE','WELL_MODE'
                 ]].copy()

well_hdr.rename(columns={'WELL_ID': 'well_id',
                         'X': 'x','Y':'y',
                         'WELL_NAME': 'well_name_original',
                         'LICENCE_NUMBER': 'license_number',
                         'TOTAL_DEPTH': 'total_depth',
                         'SPUD_DATE': 'spud_date',
                         'OPERATOR': 'operator',
                         'WELL_TYPE': 'well_type',
                         'WELL_MODE':'well_mode'}, inplace=True)

In [46]:
well_utm=[]
for x in range(len(well_hdr)):
    well_utm.append(7)
    
well_utm=pd.DataFrame(well_utm)

In [47]:
well_hdr['utm']=well_utm
well_hdr

,well_id,x,y,well_name_original,license_number,total_depth,spud_date,operator,well_type,well_mode,utm
0,32702,-80.218198,43.133536,Bow Park Farm Well #9,F020287,192.33,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7
1,32703,-80.206309,43.121120,Bow Park Farm Well #3,F020288,186.23,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7
2,32704,-80.213643,43.113897,Bow Park Farm Well #6,F020289,197.51,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7
3,32705,-80.337281,43.177397,Brant Hydro Station,F020290,102.11,1800/01/01 00:00:00+00,UNKNOWN,Dry Hole,Unknown,7
4,32706,-80.209004,43.133897,Bow Park Well #7,F020291,187.76,1800/01/01 00:00:00+00,BOW PARK FARMS,Gas Show,Unknown,7
...,...,...,...,...,...,...,...,...,...,...,...
26661,9034,-82.159344,42.110886,C.W.P. - L.E. MLO No.25,T000165,412.39,1959/08/06 00:00:00+00,LAGASCO INC.,Natural Gas Well,Active Well,7
26662,9035,-82.161011,42.101720,C.W.P. - L.E. MLO No.26,T000169,413.31,1959/09/13 00:00:00+00,LAGASCO INC.,Natural Gas Well,Active Well,7
26663,9036,-82.094620,42.133109,C.W.P. - L.E. MLO No.27,T000535,425.50,1960/06/07 00:00:00+00,DUNDEE OIL AND GAS LIMITED,Natural Gas Well,Abandoned Well,7
26664,9037,-79.872912,42.944242,(private well name),T005777,224.03,1981/11/30 00:00:00+00,(PRIVATE OPERATOR),Private Gas Well,Active Well,7


In [48]:
well_num=[]
for x in range(len(well_hdr)):
    well= well_hdr['well_name_original'][x]
    try:
        n=well.split('#')[1]
    except:
        n=0
    try:
        num=n.split(' ')[0]
    except:
        num=0   
    well_num.append(num)    
        

In [49]:
well_hdr['well_number']= well_num

In [50]:
well_name=[]
for x in range(len(well_hdr)):
    well= well_hdr['well_name_original'][x]
    try:
        n=well.split('#')[0]
    except:
        n=well
    n=n.rstrip()
    well_name.append(n)

In [51]:
well_hdr['well_name']= well_name

In [35]:
well_hdr

,well_id,x,y,well_name_original,license_number,total_depth,spud_date,operator,well_type,well_mode,utm,well_number,well_name
0,32702,-80.218198,43.133536,Bow Park Farm Well #9,F020287,192.33,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7,9,Bow Park Farm Well
1,32703,-80.206309,43.121120,Bow Park Farm Well #3,F020288,186.23,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7,3,Bow Park Farm Well
2,32704,-80.213643,43.113897,Bow Park Farm Well #6,F020289,197.51,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7,6,Bow Park Farm Well
3,32705,-80.337281,43.177397,Brant Hydro Station,F020290,102.11,1800/01/01 00:00:00+00,UNKNOWN,Dry Hole,Unknown,7,0,Brant Hydro Station
4,32706,-80.209004,43.133897,Bow Park Well #7,F020291,187.76,1800/01/01 00:00:00+00,BOW PARK FARMS,Gas Show,Unknown,7,7,Bow Park Well
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26661,9034,-82.159344,42.110886,C.W.P. - L.E. MLO No.25,T000165,412.39,1959/08/06 00:00:00+00,LAGASCO INC.,Natural Gas Well,Active Well,7,0,C.W.P. - L.E. MLO No.25
26662,9035,-82.161011,42.101720,C.W.P. - L.E. MLO No.26,T000169,413.31,1959/09/13 00:00:00+00,LAGASCO INC.,Natural Gas Well,Active Well,7,0,C.W.P. - L.E. MLO No.26
26663,9036,-82.094620,42.133109,C.W.P. - L.E. MLO No.27,T000535,425.50,1960/06/07 00:00:00+00,DUNDEE OIL AND GAS LIMITED,Natural Gas Well,Abandoned Well,7,0,C.W.P. - L.E. MLO No.27
26664,9037,-79.872912,42.944242,(private well name),T005777,224.03,1981/11/30 00:00:00+00,(PRIVATE OPERATOR),Private Gas Well,Active Well,7,0,(private well name)


In [36]:
well_hdr = pd.merge(left= well_hdr, right=cat_well_mode, left_on='well_mode', right_on='well_mode', how='left')

In [38]:
well_hdr = pd.merge(left= well_hdr, right=cat_operator, left_on='operator', right_on='operator_name', how='left')

In [52]:
well_hdr = pd.merge(left= well_hdr, right=cat_well_type, left_on='well_type', right_on='well_type', how='left')
well_hdr

,well_id,x,y,well_name_original,license_number,total_depth,spud_date,operator,well_type,well_mode,utm,well_number,well_name,well_type_id
0,32702,-80.218198,43.133536,Bow Park Farm Well #9,F020287,192.33,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7,9,Bow Park Farm Well,0
1,32703,-80.206309,43.121120,Bow Park Farm Well #3,F020288,186.23,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7,3,Bow Park Farm Well,0
2,32704,-80.213643,43.113897,Bow Park Farm Well #6,F020289,197.51,1904/01/01 00:00:00+00,BOW PARK FARMS,Natural Gas Well,Unknown,7,6,Bow Park Farm Well,0
3,32705,-80.337281,43.177397,Brant Hydro Station,F020290,102.11,1800/01/01 00:00:00+00,UNKNOWN,Dry Hole,Unknown,7,0,Brant Hydro Station,1
4,32706,-80.209004,43.133897,Bow Park Well #7,F020291,187.76,1800/01/01 00:00:00+00,BOW PARK FARMS,Gas Show,Unknown,7,7,Bow Park Well,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26661,9034,-82.159344,42.110886,C.W.P. - L.E. MLO No.25,T000165,412.39,1959/08/06 00:00:00+00,LAGASCO INC.,Natural Gas Well,Active Well,7,0,C.W.P. - L.E. MLO No.25,0
26662,9035,-82.161011,42.101720,C.W.P. - L.E. MLO No.26,T000169,413.31,1959/09/13 00:00:00+00,LAGASCO INC.,Natural Gas Well,Active Well,7,0,C.W.P. - L.E. MLO No.26,0
26663,9036,-82.094620,42.133109,C.W.P. - L.E. MLO No.27,T000535,425.50,1960/06/07 00:00:00+00,DUNDEE OIL AND GAS LIMITED,Natural Gas Well,Abandoned Well,7,0,C.W.P. - L.E. MLO No.27,0
26664,9037,-79.872912,42.944242,(private well name),T005777,224.03,1981/11/30 00:00:00+00,(PRIVATE OPERATOR),Private Gas Well,Active Well,7,0,(private well name),10


In [42]:
well_hdr_df=well_hdr[['well_id', 'x', 'y', 'utm', 'well_name', 'well_number', 'operator_id', 'well_type_id','well_mode_id', 'license_number', 'total_depth', 'spud_date' ]]

In [23]:
well_hdr_df

,well_id,x,y,utm,well_name,well_number,operator_id,well_type_id,well_mode_id,licence_number,total_depth,spud_date
0,32702,-80.218198,43.133536,7,Bow Park Farm Well,9,0,0,0,F020287,192.33,1904/01/01 00:00:00+00
1,32703,-80.206309,43.121120,7,Bow Park Farm Well,3,0,0,0,F020288,186.23,1904/01/01 00:00:00+00
2,32704,-80.213643,43.113897,7,Bow Park Farm Well,6,0,0,0,F020289,197.51,1904/01/01 00:00:00+00
3,32705,-80.337281,43.177397,7,Brant Hydro Station,0,1,1,0,F020290,102.11,1800/01/01 00:00:00+00
4,32706,-80.209004,43.133897,7,Bow Park Well,7,0,2,0,F020291,187.76,1800/01/01 00:00:00+00
...,...,...,...,...,...,...,...,...,...,...,...,...
26661,9034,-82.159344,42.110886,7,C.W.P. - L.E. MLO No.25,0,48,0,3,T000165,412.39,1959/08/06 00:00:00+00
26662,9035,-82.161011,42.101720,7,C.W.P. - L.E. MLO No.26,0,48,0,3,T000169,413.31,1959/09/13 00:00:00+00
26663,9036,-82.094620,42.133109,7,C.W.P. - L.E. MLO No.27,0,47,0,1,T000535,425.50,1960/06/07 00:00:00+00
26664,9037,-79.872912,42.944242,7,(private well name),0,37,10,3,T005777,224.03,1981/11/30 00:00:00+00


In [43]:
rds_connection_string = "postgres:{pw}@localhost:5432/Ontario_Wells"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [26]:
engine.table_names()

['cat_operator',
 'cat_well_type',
 'cat_well_mode',
 'cat_log_type',
 'log_hdr',
 'well_hdr']

In [ ]:
cat_well_type.to_sql(name='cat_well_type', con=engine, if_exists='append', index=False)

In [ ]:
cat_well_mode.to_sql(name='cat_well_mode', con=engine, if_exists='append', index=False)

In [ ]:
cat_operator.to_sql(name='cat_operator', con=engine, if_exists='append', index=False)

In [44]:
well_hdr_df.to_sql(name='well_hdr', con=engine, if_exists='append', index=False)